In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/get_started"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Acesse ai.google.dev</a>
  </td>
    <td>
    <a target="_blank" href="https://colab.research.google.com/github.com/lucianommartins/generative-ai-ptbr/blob/main/gemma/1_intro_gemma.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Execute este notebook no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/lucianommartins/generative-ai-ptbr/blob/main/gemma/1_intro_gemma.ipynb"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />Execute este notebook na Vertex AI</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/lucianommartins/generative-ai-ptbr/blob/main/gemma/1_intro_gemma.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Consulte o código no GitHub</a>
  </td>
</table>

# Primeiros passos com Gemma utilizando KerasNLP

Este tutorial mostra como começar a usar Gemma usando [KerasNLP](https://keras.io/keras_nlp/). Gemma é uma família de modelos abertos leves e de última geração, construídos a partir da mesma pesquisa e tecnologia usada para criar os modelos Gemini. KerasNLP é uma coleção de modelos de processamento de linguagem natural (PNL) implementados em [Keras](https://keras.io/) e executáveis em JAX, PyTorch e TensorFlow.

Neste tutorial, você usará o Gemma para gerar respostas de texto para vários prompts. Se você é novo no Keras, talvez queira ler [Introdução ao Keras](https://keras.io/getting_started/) antes de começar, mas não é necessário. Você aprenderá mais sobre Keras à medida que avança neste tutorial.

**Importante:** Para uma melhor experiência de execução deste laboratório, é recomendado que se utilize uma máquina com aceleradores (GPU ou TPU). Em uma máquina sem aceleradores (como GPU) esta geração de conteúdo pode levar cerca de 80 segundos para ser concluída.

## Configurando o ambiente

### Configurações para o uso do Gemma

Para concluir este tutorial, primeiro você precisa seguir as instruções de configuração em [Configuração do Gemma](https://ai.google.dev/gemma/docs/setup). As instruções de configuração do Gemma mostram como fazer o seguinte:

* Obtenha acesso a Gemma em kaggle.com.
* Selecione um tempo de execução do Colab com recursos suficientes para execução
   o modelo Gemma 2B.
* Gere e configure um nome de usuário Kaggle e uma chave de API.

Depois de concluir a configuração do Gemma, vá para a próxima seção, onde você definirá variáveis de ambiente para seu ambiente Colab.

**TL;DR: Acesse https://www.kaggle.com/models/google/gemma e faça a solicitação de acesso ao modelo Gemma**

### Instalando dependências

Instale os pacotes `pip` Keras e KerasNLP.

In [ ]:
# Instale a última versão do Keras 3. Acesse https://keras.io/getting_started/ para maiores detalhes.
!pip install -U keras-nlp
!pip install -U keras>=3
!pip install -U jax
!pip install -U jaxlib

### Configurando variáveis de ambiente necessárias para utilizar o Kaggle

Configure as variáveis `KAGGLE_USERNAME` (contendo seu usuário Kaggle) e `KAGGLE_KEY` (com sua chave de acesso ao Kaggle).

In [ ]:
import os

# Nota: `userdata.get` é uma API Colab. Se você não estiver usando o Colab, defina o ambiente
# vars conforme apropriado para seu sistema.

# descomente o par de linhas que faça mais sentido pra você:
# # opção A: utilizando direto no Colab
# from google.colab import userdata
# os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
# os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

# # opção B: utilizando na Vertex AI ou no seu ambiente pessoal
# os.environ["KAGGLE_USERNAME"] = "seu usuário"
# os.environ["KAGGLE_KEY"] = "sua chave de autenticação"

### Escolhe o backend a ser utilizado com o Keras

Keras é uma API de deep learning multi-backend de alto nível projetada para simplicidade e facilidade de uso. [Keras 3](https://keras.io/keras_3) permite escolher o backend: TensorFlow, JAX ou PyTorch. Todos os três funcionarão para este tutorial.

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Ou "tensorflow" ou "torch".

### Importando os módulos necessários

Importe o Keras e o KerasNLP.

In [ ]:
import keras
import keras_nlp

## Defina um modelo

KerasNLP fornece implementações de muitas [arquiteturas de modelos populares](https://keras.io/api/keras_nlp/models/). Neste tutorial, você criará um modelo usando `GemmaCausalLM`, um modelo Gemma ponta a ponta para modelagem de linguagem causal. Um modelo de linguagem causal prevê o próximo token com base nos tokens anteriores.

Crie o modelo usando o método `from_preset`:

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

`from_preset` instancia o modelo a partir de uma arquitetura e pesos predefinidos. No código acima, a string `"gemma_2b_en"` especifica a arquitetura predefinida: um modelo Gemma com 2 bilhões de parâmetros.

Nota: Também está disponível um modelo Gemma com 7 bilhões de parâmetros. Para executar o modelo maior no Colab, você precisa de acesso às GPUs premium disponíveis em planos pagos. Como alternativa, você pode realizar [ajuste distribuído em um modelo Gemma 7B](https://ai.google.dev/gemma/docs/distributed_tuning) no Kaggle ou no Google Cloud.

Use `summary` para obter mais informações sobre o modelo:

In [ ]:
gemma_lm.summary()

Como você pode ver no resumo, o modelo possui 2,5 bilhões de parâmetros treináveis.

Nota: Para fins de nomeação do modelo ("2B"), a camada de embeddings não é contabilizada no número de parâmetros.

## Gerando textos

Agora é hora de gerar algum texto! O modelo possui um método `generate` que gera texto com base em um prompt. O argumento opcional `max_length` especifica o comprimento máximo da sequência gerada.

Experimente com a pergunta `"Qual é o sentido da vida?"`.

In [ ]:
gemma_lm.generate("Qual é o sentido da vida?", max_length=64)

Tente chamar `generate` novamente com um prompt diferente.

In [ ]:
gemma_lm.generate("Como o cérebro funciona?", max_length=64) # altere o valor do prompt para as perguntas que você quiser

Se você estiver executando back-ends JAX ou TensorFlow, notará que a segunda chamada `generate` retorna quase instantaneamente. Isso ocorre porque cada chamada para `generate` para um determinado tamanho de lote e `max_length` é compilada com XLA. A primeira execução é cara, mas as execuções subsequentes são muito mais rápidas.

Você pode também utilizar prompts em "batch", utilizando uma lista como entrada:

In [ ]:
gemma_lm.generate(
    ["Qual é o sentido da vida?",
     "Como o cérebro funciona?"],
    max_length=64)

### Opcional: experimente um sampler diferente

Você pode controlar a estratégia de geração para `GemmaCausalLM` definindo o argumento `sampler` em `compile()`. Por padrão, a amostragem [`"greedy"`](https://keras.io/api/keras_nlp/samplers/greedy_sampler/#greedysampler-class) será usada.

Como experiência, tente definir uma estratégia [`"top_k"`](https://keras.io/api/keras_nlp/samplers/top_k_sampler/):

In [ ]:
gemma_lm.compile(sampler="top_k")
gemma_lm.generate("Qual o sentido da vida?", max_length=64)

Enquanto o algoritmo `"greedy"` padrão sempre escolha o token com a maior probabilidade, o algoritmo top-K escolhe aleatoriamente o próximo token entre os tokens com a maior probabilidade K.

Você não precisa especificar um sampler e pode ignorar o último trecho de código se ele não for útil para seu caso de uso. Se quiser saber mais sobre os samplers disponíveis, consulte [Samplers](https://keras.io/api/keras_nlp/samplers/).

## Próximos passos

Neste tutorial, você aprendeu como gerar texto usando KerasNLP e Gemma. Aqui estão algumas sugestões sobre o que aprender a seguir:

* Aprenda como [realizar tuning de um modelo Gemma](https://ai.google.dev/gemma/docs/lora_tuning).
* Aprenda como realizar [tuning distribuído e inferência em um modelo Gemma](https://ai.google.dev/gemma/docs/distributed_tuning).
* Saiba mais sobre [integração do Gemma com Vertex AI](https://ai.google.dev/gemma/docs/integrations/vertex)
* Saiba como [usar modelos Gemma com Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma).